In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate 
from tools.chord_extractor import chord_extractor
from tools.chord_sheet_getter import chord_sheet_getter

ImportError: attempted relative import with no known parent package

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', openai_api_key = ${{ secrets.API_KEY }}, temperature = 0)

# List the tools that AI can use
tools = [chord_sheet_getter, chord_extractor]

# Set up the Agent, give it the tools
agent = initialize_agent(tools,
                         llm,
                         agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True,
                         max_iterations = 2)


prompt = PromptTemplate(input_variables = ['song_name', 'artist_name'],                            
          template = '''
            You are an AI assistant who's been given access to tools to create urls to mychordbook.com, pull unclean chord sheet data from mychordbook.com and then clean up the unclean data and return it in a desired format.
            Please get the chords for the song {song_name} by {artist_name} and clean them.         
           '''                                                                      
        )


# Run the agent with the provided input variable
result = agent.run(prompt.format_prompt(song_name = 'smells like teen spirit', artist_name = 'nirvana'))
print(result)


